## Implicit collaborative filtering approaches - pseudo product index

### Background

This is the development pipeline to apply the implicit item-based recommendation system algorithms for gracioustyle customers. The aim is to test and compare a few algorithms including  
  
* Alternating Least Square
* Bayesian Personalized Ranking
  
The approaches above can be built into a hybrid CF approach, and further combined with the content based apporach as the optimized strategy. 

We modified the modelling level to a pseudo product index concatenating product brand, type and color, instead of the actual product level. 

### Import libraries

In [4]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import random
import implicit
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize

from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.datasets.movielens import get_movielens
from implicit.lmf import LogisticMatrixFactorization
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight,tfidf_weight)
import tqdm
import codecs
from tabulate import tabulate

### Import data

Order data has been merged with product table to build a master source data in json/csv format.

In [5]:
retail_data = pd.read_json('C:/Projects/DJ_projects/FP-growth/recommendation_engine/development/hybrid/data/cus_user_quality_date_0716.json') # This may take a couple minutes

In [6]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120442 entries, 0 to 120441
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CUSTOMER_DIM_KEY   120442 non-null  int64  
 1   PRODUCT_DIM_KEY    120442 non-null  int64  
 2   QUANTITY           120441 non-null  float64
 3   PRODUCT_NAME       120435 non-null  object 
 4   COLOR              120442 non-null  object 
 5   PRODUCT_TYPE       120442 non-null  object 
 6   CATEGORY           120442 non-null  object 
 7   BRAND              120442 non-null  object 
 8   RESOLUTION_STATUS  120442 non-null  object 
 9   ORDER_DATE_KEY     120442 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 9.2+ MB


In [7]:
retail_data.head()

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,QUANTITY,PRODUCT_NAME,COLOR,PRODUCT_TYPE,CATEGORY,BRAND,RESOLUTION_STATUS,ORDER_DATE_KEY
0,60189,128674,2.0,Montana blanket sham european,Unknown,"blankets,shams",MONTANA-BLANKET,Pine Cone Hill,COMPLETED,17178
1,60189,128673,1.0,Montana blanket full/queen,Unknown,blankets,MONTANA-BLANKET,Pine Cone Hill,COMPLETED,17178
2,57104,336148,1.0,Sago Palm Bread Plate Gold,Unknown,bread & butter plates,MICHAEL-ARAM,Michael Aram,COMPLETED,17178
3,55765,320505,0.0,Galletto Red Condiment Bowl,Unknown,condiment holders,GALLETTO-DINNER,Vietri,REJECTED,17178
4,60157,181252,1.0,Butterfly Garden Bread & Butter Plate 7 inch,Unknown,bread & butter plates,TABLE-SETTINGS-YELLO,Rosenthal,COMPLETED,17177


In [8]:
retail_data.count()

CUSTOMER_DIM_KEY     120442
PRODUCT_DIM_KEY      120442
QUANTITY             120441
PRODUCT_NAME         120435
COLOR                120442
PRODUCT_TYPE         120442
CATEGORY             120442
BRAND                120442
RESOLUTION_STATUS    120442
ORDER_DATE_KEY       120442
dtype: int64

Build a pseudo product index combining the product type, brand and color

In [9]:
prod_cols = ['PRODUCT_TYPE', 'BRAND', 'COLOR']
retail_data['PROD_INDEX'] = retail_data[prod_cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [10]:
retail_data.head()

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,QUANTITY,PRODUCT_NAME,COLOR,PRODUCT_TYPE,CATEGORY,BRAND,RESOLUTION_STATUS,ORDER_DATE_KEY,PROD_INDEX
0,60189,128674,2.0,Montana blanket sham european,Unknown,"blankets,shams",MONTANA-BLANKET,Pine Cone Hill,COMPLETED,17178,"blankets,shams_Pine Cone Hill_Unknown"
1,60189,128673,1.0,Montana blanket full/queen,Unknown,blankets,MONTANA-BLANKET,Pine Cone Hill,COMPLETED,17178,blankets_Pine Cone Hill_Unknown
2,57104,336148,1.0,Sago Palm Bread Plate Gold,Unknown,bread & butter plates,MICHAEL-ARAM,Michael Aram,COMPLETED,17178,bread & butter plates_Michael Aram_Unknown
3,55765,320505,0.0,Galletto Red Condiment Bowl,Unknown,condiment holders,GALLETTO-DINNER,Vietri,REJECTED,17178,condiment holders_Vietri_Unknown
4,60157,181252,1.0,Butterfly Garden Bread & Butter Plate 7 inch,Unknown,bread & butter plates,TABLE-SETTINGS-YELLO,Rosenthal,COMPLETED,17177,bread & butter plates_Rosenthal_Unknown


In [11]:
retail_data.groupby('PROD_INDEX')['QUANTITY'].agg('sum').sort_values(ascending=False)

PROD_INDEX
drinking glasses,bath tumblers_Mario Luca Giusti_Unknown    3651.0
napkins_Garnier-Thiebaut_Unknown                            3546.0
table protectors_Gracious Style_White                       3515.0
Unknown_Gracious Style_Unknown                              2706.0
placemats_Kim Seybert_Unknown                               2361.0
                                                             ...  
pillow cases_Gracious Style_White/Pink                        -4.0
salad & dessert plates_Pom Pom at Home_Unknown                -6.0
dinner plates_Pom Pom at Home_Unknown                         -8.0
cereal & soup bowls_Pom Pom at Home_Unknown                   -8.0
Unknown_Anali_Cream                                          -20.0
Name: QUANTITY, Length: 8802, dtype: float64

Some data subseting has been done based on the following conditions,  
  
* Only contains the order history on products that have had orders since 2018. This is used as a proxy as the current product catelogue. 
* Resolution status in 'Completed' or 'Pending'.
* De-duplicate the identical rows.

In [12]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.PRODUCT_NAME) == False]
cleaned_retail = cleaned_retail[cleaned_retail['PRODUCT_DIM_KEY'].isin(cleaned_retail[cleaned_retail['ORDER_DATE_KEY'] > 17533].PRODUCT_DIM_KEY)]
cleaned_retail['QUANTITY'] = cleaned_retail['QUANTITY'].fillna(0.0)
cleaned_retail = cleaned_retail[cleaned_retail.RESOLUTION_STATUS.isin(['COMPLETED', 'PENDING'])]
print('Duplicated rows: ' + str(cleaned_retail.duplicated().sum()))
cleaned_retail.drop_duplicates(inplace=True)

Duplicated rows: 280


In [13]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51810 entries, 0 to 120440
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CUSTOMER_DIM_KEY   51810 non-null  int64  
 1   PRODUCT_DIM_KEY    51810 non-null  int64  
 2   QUANTITY           51810 non-null  float64
 3   PRODUCT_NAME       51810 non-null  object 
 4   COLOR              51810 non-null  object 
 5   PRODUCT_TYPE       51810 non-null  object 
 6   CATEGORY           51810 non-null  object 
 7   BRAND              51810 non-null  object 
 8   RESOLUTION_STATUS  51810 non-null  object 
 9   ORDER_DATE_KEY     51810 non-null  int64  
 10  PROD_INDEX         51810 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 4.7+ MB


In [14]:
cleaned_retail['PROD_INDEX'].value_counts()

table protectors_Gracious Style_White    2602
detergents_Le Blanc_Unknown              1096
red wine_Zalto_Unknown                    573
tissue boxes_Pigeon & Poodle_Unknown      565
rugs_Dash & Albert_Unknown                531
                                         ... 
fitted sheets_Kassatex_Navy                 1
napkins_Sferra_Flamingo                     1
cake platters & stands_Nude_Unknown         1
rugs,samples_loloi_Terracotta               1
vases,pitchers_Arte Italica_Unknown         1
Name: PROD_INDEX, Length: 3719, dtype: int64

### Threhold function for cut-off

In [15]:
def threshold_likes(df, uid_min, mid_min, user_column = 'CUSTOMER_DIM_KEY', item_column = 'PROD_INDEX'):
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby(user_column)[item_column].count()
        df = df[~df[user_column].isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby(item_column)[user_column].count()
        df = df[~df[item_column].isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby(user_column)[item_column].count().min() >= mid_min)
    assert(df.groupby(item_column)[user_column].count().min() >= uid_min)
    
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

### Product lookup table

In [16]:
df_lim = threshold_likes(cleaned_retail, 2, 2)
item_lookup = df_lim[['PROD_INDEX']].drop_duplicates() # Only get unique item/description pairs
item_lookup['PROD_INDEX'] = item_lookup.PROD_INDEX.astype(str) # Encode as strings for future lookup ease

Starting likes info
Number of users: 27143
Number of models: 3719
Sparsity: 0.051%
Ending likes info
Number of users: 9011
Number of models: 2085
Sparsity: 0.174%


### User-product matrix function

In [17]:
def create_matrix(cleaned_retail, row_feature, col_feature):
    # cleaned_retail[row_feature] = cleaned_retail[row_feature].astype(int) # Convert to int for customer ID
    cleaned_retail = cleaned_retail[[col_feature, 'QUANTITY', row_feature]] # Get rid of unnecessary info
    grouped_cleaned = cleaned_retail.groupby([row_feature, col_feature]).sum().reset_index() # Group together
    grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
    # indicate purchased
    grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive
    
    customers = list(np.sort(grouped_purchased[row_feature].unique())) # Get our unique customers
    products = list(np.sort(grouped_purchased[col_feature].unique())) # Get our unique products that were purchased
    quantity = list(grouped_purchased.QUANTITY) # All of our purchases
    #print(products)
    rows = grouped_purchased[row_feature].astype('category', CategoricalDtype(categories = customers)).cat.codes 
    # Get the associated row indices
    cols = grouped_purchased[col_feature].astype('category', CategoricalDtype(categories = products)).cat.codes 
    # Get the associated column indices
    purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))
    return purchases_sparse, customers, products

### Create user-product matrix

In [18]:
purchases_sparse, products, customers = create_matrix(df_lim, 'PROD_INDEX', 'CUSTOMER_DIM_KEY')

C:\Users\david_qi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [19]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [20]:
products_arr

array(['Unknown_Abbiamo Tutto_Unknown', 'Unknown_Alessi_Unknown',
       'Unknown_Anna Weatherley_Unknown', ...,
       'wine coolers_Mario Luca Giusti_Unknown',
       'wine coolers_Match_Unknown', 'wine coolers_Sambonet_Unknown'],
      dtype='<U114')

In [21]:
len(set(customers_arr))

9011

In [22]:
len(set(products_arr))

2085

### Train and test data prep

In [23]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [24]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

### Function to calculate AUC (evaluation metric)

In [25]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
def calc_mean_auc_cv(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
def calc_mean_auc_pop(training_set, altered_users, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    
    return float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1],
              'alpha': [1, 10, 50, 100]}

def cross_validation_als(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    #product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)
    #product_train, product_test, product_users_altered = make_train(product_train, pct_test = 0.2)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9)).tocsr()

    model = AlternatingLeastSquares()
    pop_auc = calc_mean_auc_pop(product_train, product_users_altered, product_test)

    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_model = model
    print('The mean AUC of popularity benchmark: ', pop_auc)
    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for alpha in param_grid['alpha']:
                model = AlternatingLeastSquares(factors = factor, regularization = regu)
                model.fit(training_set*alpha)
                item_vecs = model.user_factors
                user_vecs = model.item_factors
                store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                print(factor, regu, alpha, store_auc)
                
                if store_auc > best_auc:
                    best_auc = store_auc
                    best_factors = factor
                    best_reg = regu
                    best_alpha = alpha
                    best_model = model
                    
    print('Best AUC: ', best_auc)
    print('Pop AUC: ', pop_auc)
    print('Best factor: ', best_factors)
    print('Best regularization: ', best_reg)
    print('Best alpha: ', best_alpha)
    return best_auc, pop_auc, best_factors, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

### ALS hyper tuning with cross validation

In [26]:
cv_als = cross_validation_als(purchases_sparse, 'als', param_grid)

The mean AUC of popularity benchmark:  0.687



10 0.0 1 0.73



10 0.0 10 0.765



10 0.0 50 0.768



10 0.0 100 0.768


...


120 0.001 100 0.754



120 0.1 1 0.78



120 0.1 10 0.779



120 0.1 50 0.767



120 0.1 100 0.76
Best AUC:  0.803
Pop AUC:  0.687
Best factor:  40
Best regularization:  1e-05
Best alpha:  10


In [27]:
cv_als[4]

10

### ALS hyper tuning distance parameters

In [28]:
model = AlternatingLeastSquares(factors = cv_als[2], regularization = cv_als[3])
# The literature suggests values between 0 and 3 for K1 and 0 and 1 for B. 
ratings = bm25_weight(product_train, K1 = 0.1, B = 1).tocsr()
model.fit(ratings)
item_vecs = model.user_factors
user_vecs = model.item_factors
store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
store_auc, pop_auc

(0.785, 0.687)

In [29]:
param_grid = {'B': [0.1, 0.5, 1, 2, 3],
              'K1': [0, 0.1, 0.01, 1]
             }

def cross_validation_als_bm25_param(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    #product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)
    #product_train, product_test, product_users_altered = make_train(product_train, pct_test = 0.2)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9)).tocsr()

    model = AlternatingLeastSquares()
    pop_auc = calc_mean_auc_pop(product_train, product_users_altered, product_test)
    best_auc=0
    best_B = 0
    best_K1 = 0
    best_model = model
    print('The mean AUC of popularity benchmark: ', pop_auc)
    for B in param_grid['B']:
        for K1 in param_grid['K1']:
            training_set = (bm25_weight(product_train, B=B,K1=K1)).tocsr()
            model = AlternatingLeastSquares(factors = cv_als[2], regularization = cv_als[3])
            model.fit(training_set)
            item_vecs = model.user_factors
            user_vecs = model.item_factors
            store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
            print(B, K1, store_auc)
                
            if store_auc > best_auc:
                best_auc = store_auc
                best_B = B
                best_K1 = K1
                best_model = model
                    
    print('Best AUC: ', best_auc)
    print('Pop AUC: ', pop_auc)
    print('Best B: ', best_B)
    print('Best K1: ', best_K1)
    return best_auc, pop_auc, best_B, best_K1, best_model
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [30]:
cv_als_w = cross_validation_als_bm25_param(purchases_sparse, 'als', param_grid)

The mean AUC of popularity benchmark:  0.687



0.1 0 0.776



0.1 0.1 0.779



0.1 0.01 0.773



0.1 1 0.784


...


2 1 0.787



3 0 0.771



3 0.1 0.793



3 0.01 0.774



3 1 0.7
Best AUC:  0.799
Pop AUC:  0.687
Best B:  1
Best K1:  1


### Best ALS model

Select the best ALS model from the hyper tuning process.

In [31]:
best_als = cv_als_w[4]

### BPR-MF hyper tuning with cross validation

In [32]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001]
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [0.1],
              'regularization': [0.0, 0.1,0.01,0.001],
              'alpha': [1, 10, 50, 100, 500]}

def cross_validation_bpr(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    if model_name == "bpr":
        model = BayesianPersonalizedRanking()
    if model_name == "lmf":
        model = LogisticMatrixFactorization()
        
    training_set = product_train
    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_learning_rate = 0

    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for lr in param_grid['learning_rate']:
                for alpha in param_grid['alpha']:
                    model = BayesianPersonalizedRanking(factors = factor,learning_rate= lr, regularization = regu)
                    model.fit(training_set*alpha)
                    item_vecs = model.user_factors
                    user_vecs = model.item_factors
                    store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                    print(factor, regu, lr, alpha, store_auc, pop_auc)

                    if store_auc > best_auc:
                        best_auc = store_auc
                        best_factors = factor
                        best_reg = regu
                        best_alpha = alpha
                        best_learning_rate = lr
                        best_model = model
    print(best_auc, best_factors, best_learning_rate, best_reg, best_alpha)
    return best_auc, best_factors,best_learning_rate, best_reg, best_alpha, best_model
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [33]:
cv_bpr = cross_validation_bpr(purchases_sparse, 'bpr', param_grid)


10 0.0 0.1 1 0.749 0.687



10 0.0 0.1 10 0.745 0.687



10 0.0 0.1 50 0.749 0.687



10 0.0 0.1 100 0.748 0.687



10 0.0 0.1 500 0.744 0.687


...


120 0.001 0.1 1 0.781 0.687



120 0.001 0.1 10 0.785 0.687



120 0.001 0.1 50 0.783 0.687



120 0.001 0.1 100 0.782 0.687



120 0.001 0.1 500 0.789 0.687
0.809 80 0.1 0.1 100


### Best BPR-MF model

Select the best BPR model from the hyper tuning process.

In [34]:
best_bpr = cv_bpr[5]

In [35]:
customers_arr[5000:5100]

array([68670, 68671, 68672, 68682, 68684, 68688, 68689, 68691, 68694,
       68699, 68701, 68708, 68714, 68715, 68717, 68718, 68729, 68732,
       68734, 68738, 68742, 68744, 68746, 68747, 68749, 68755, 68758,
       68760, 68762, 68763, 68765, 68768, 68770, 68772, 68775, 68780,
       68783, 68785, 68791, 68794, 68798, 68799, 68806, 68809, 68812,
       68815, 68817, 68819, 68830, 68833, 68834, 68840, 68845, 68846,
       68848, 68851, 68852, 68853, 68856, 68861, 68867, 68868, 68869,
       68870, 68874, 68879, 68882, 68883, 68885, 68886, 68887, 68888,
       68890, 68895, 68897, 68898, 68901, 68903, 68908, 68914, 68916,
       68923, 68928, 68930, 68933, 68935, 68937, 68951, 68953, 68954,
       68956, 68960, 68962, 68963, 68965, 68966, 68967, 68970, 68971,
       68972], dtype=int64)

### Get actual purchase vs model recommendations

In [36]:
def get_rec_items(model, customer_id, sparse_matrix, customer_list, item_list, item_lookup, num_items = 10):
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = sparse_matrix[:,cust_ind].nonzero()[0] # Get column indices of purchased items
    prod_codes = item_list[purchased_ind] # Get the stock codes for our purchased items
    print(prod_codes)
    purchased_items = item_lookup.loc[item_lookup.PROD_INDEX.isin(prod_codes)]
    
    print('-------------------------------------------------------------------------------------')
    print('Print purchased items in train set')
    print('-------------------------------------------------------------------------------------')
    print(tabulate(purchased_items))
    print('-------------------------------------------------------------------------------------')
    print('Print full actual purchases above threshold')
    print('-------------------------------------------------------------------------------------')
    print(tabulate(df_lim[df_lim['CUSTOMER_DIM_KEY'] == customer_id]))
    print('-------------------------------------------------------------------------------------')
    print('Print model recommendations')
    print('-------------------------------------------------------------------------------------')
    
    recom = model.recommend(cust_ind, sparse_matrix.T.tocsr(),filter_items=purchased_ind.tolist(), N= num_items)
    rec_keys = [] # start empty list to store items
    descriptions = []
    for index in recom:
        key = item_list[index[0]]
        rec_keys.append(key)
        # descriptions.append(item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == key].iloc[0])
        
    scores = [item[1] for item in recom]
    final_frame = pd.DataFrame({'PROD_INDEX': rec_keys, 'SCORE': scores}) # Create a dataframe 
    final_frame[['PROD_INDEX','SCORE']]
    return final_frame[['PROD_INDEX','SCORE']] # Switch order of columns around

In [37]:
get_rec_items(best_als, 55974, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

['Unknown_Anna Weatherley_Unknown'
 'bread & butter plates_Anna Weatherley_Unknown'
 'dinner plates_Anna Weatherley_Unknown'
 'jewelry boxes_Anna Weatherley_Unknown'
 'salad & dessert plates_Anna Weatherley_Unknown'
 'serving dishes_Anna Weatherley_Unknown']
-------------------------------------------------------------------------------------
Print purchased items in train set
-------------------------------------------------------------------------------------
-----  ----------------------------------------------
  484  dinner plates_Anna Weatherley_Unknown
  490  serving dishes_Anna Weatherley_Unknown
 8767  salad & dessert plates_Anna Weatherley_Unknown
10066  bread & butter plates_Anna Weatherley_Unknown
10816  Unknown_Anna Weatherley_Unknown
11050  jewelry boxes_Anna Weatherley_Unknown
-----  ----------------------------------------------
-------------------------------------------------------------------------------------
Print full actual purchases above threshold
--------------

,PROD_INDEX,SCORE
0,coffee mugs & tea cups_Anna Weatherley_Unknown,0.742915
1,cereal & soup bowls_Anna Weatherley_Unknown,0.715831
2,"buffet & chargers,buffet & chargers_Anna Weath...",0.691304
3,cups_Anna Weatherley_Unknown,0.601291
4,dessert bowls_Anna Weatherley_Unknown,0.582401
5,dinnerware_Anna Weatherley_Unknown,0.502657
6,"dessert plates,salad & dessert plates_Anna Wea...",0.471804
7,buffet & chargers_Anna Weatherley_Unknown,0.381713
8,creamer & sugar_Anna Weatherley_Unknown,0.376902
9,Unknown_Mottahedeh_Unknown,0.272487


In [38]:
get_rec_items(best_bpr, 55974, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

['Unknown_Anna Weatherley_Unknown'
 'bread & butter plates_Anna Weatherley_Unknown'
 'dinner plates_Anna Weatherley_Unknown'
 'jewelry boxes_Anna Weatherley_Unknown'
 'salad & dessert plates_Anna Weatherley_Unknown'
 'serving dishes_Anna Weatherley_Unknown']
-------------------------------------------------------------------------------------
Print purchased items in train set
-------------------------------------------------------------------------------------
-----  ----------------------------------------------
  484  dinner plates_Anna Weatherley_Unknown
  490  serving dishes_Anna Weatherley_Unknown
 8767  salad & dessert plates_Anna Weatherley_Unknown
10066  bread & butter plates_Anna Weatherley_Unknown
10816  Unknown_Anna Weatherley_Unknown
11050  jewelry boxes_Anna Weatherley_Unknown
-----  ----------------------------------------------
-------------------------------------------------------------------------------------
Print full actual purchases above threshold
--------------

,PROD_INDEX,SCORE
0,coffee mugs & tea cups_Anna Weatherley_Unknown,1.964764
1,cereal & soup bowls_Anna Weatherley_Unknown,1.899779
2,cups_Anna Weatherley_Unknown,1.799653
3,dinnerware_Anna Weatherley_Unknown,1.760103
4,dessert bowls_Anna Weatherley_Unknown,1.695818
5,"dessert plates,salad & dessert plates_Anna Wea...",1.656197
6,creamer & sugar_Anna Weatherley_Unknown,1.626868
7,gravy boats_Anna Weatherley_Unknown,1.617269
8,placecard holders_Anna Weatherley_Unknown,1.595425
9,vases_Anna Weatherley_Unknown,1.476409


### Product index similarities

In [39]:
def get_sim_item(model, item_id):
    purchased_items = item_lookup.loc[item_lookup.PROD_INDEX.isin([item_id])]
    item_ind = np.where(products_arr == item_id)
    print(purchased_items)
    arr = model.similar_items(item_ind[0][0], N=20)
    rec_keys = [] # start empty list to store items
    descriptions = []
    for index in arr:
        key = products_arr[index[0]]
        rec_keys.append(key)
        # descriptions.append(item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == key].iloc[0])
        
    scores = [item[1] for item in arr]
    final_frame = pd.DataFrame({'PROD_INDEX': rec_keys, 'SCORE': scores}) # Create a dataframe 
    final_frame[['PROD_INDEX', 'SCORE']]
    return final_frame[['PROD_INDEX','SCORE']] # Switch order of columns around

In [40]:
item_lookup.loc[item_lookup.PROD_INDEX.isin(['blankets_Pine Cone Hill_Unknown'])]

,PROD_INDEX
1,blankets_Pine Cone Hill_Unknown


In [41]:
# Best ALS
get_sim_item(model = best_als, item_id = 'blankets_Pine Cone Hill_Unknown')

                        PROD_INDEX
1  blankets_Pine Cone Hill_Unknown


,PROD_INDEX,SCORE
0,blankets_Pine Cone Hill_Unknown,0.043979
1,"blankets,shams_Pine Cone Hill_Unknown",0.043628
2,sculpture_Arteriors_Unknown,0.042637
3,book ends_Arteriors_Unknown,0.040900
4,ice scoops_Michael Aram_Unknown,0.037337
5,"cereal & soup bowls,handle_L'Objet_Unknown",0.037250
6,bath rugs_Pine Cone Hill_Unknown,0.036833
7,bath sheets_Kassatex_Misty Sage,0.036546
8,hand towels_Kassatex_Misty Sage,0.036434
9,wash cloths_Kassatex_Misty Sage,0.035966


In [42]:
# Best BPR-MF to rank similar product indexes
get_sim_item(model = best_bpr, item_id = 'blankets_Pine Cone Hill_Unknown')

                        PROD_INDEX
1  blankets_Pine Cone Hill_Unknown


,PROD_INDEX,SCORE
0,blankets_Pine Cone Hill_Unknown,1.138527
1,"blankets,shams_Pine Cone Hill_Unknown",1.067209
2,book ends_Arteriors_Unknown,0.808184
3,sculpture_Arteriors_Unknown,0.801041
4,napkins_Pom Pom at Home_Unknown,0.490074
5,book ends_L'Objet_Unknown,0.458960
6,canisters_Kassatex_Unknown,0.443746
7,Unknown_Jamie Young_Unknown,0.412404
8,soap dispensers_Kassatex_Unknown,0.407643
9,soap dishes_Kassatex_Unknown,0.405795


In [44]:
# Best BPR-MF to rank similar product indexes
get_sim_item(model = best_bpr, item_id = 'table protectors_Gracious Style_White')

                                PROD_INDEX
154  table protectors_Gracious Style_White


,PROD_INDEX,SCORE
0,table protectors_Gracious Style_White,1.163005
1,tablecloths_Sferra_White,1.042325
2,napkins_Sferra_White,1.028270
3,pillow cases_Kassatex_White,1.017855
4,"custom tablecloths,tablecloths_Sferra_White",1.010366
5,placemats_Sferra_White,0.976188
6,shams_Sferra_Ivory,0.943366
7,napkins_Sferra_Navy,0.940504
8,samples_Sferra_Unknown,0.934027
9,tablecloths_Sferra_Stone,0.933288


In [45]:
# Best BPR-MF to rank similar product indexes
get_sim_item(model = best_bpr, item_id = 'detergents_Le Blanc_Unknown')

                      PROD_INDEX
257  detergents_Le Blanc_Unknown


,PROD_INDEX,SCORE
0,detergents_Le Blanc_Unknown,1.031987
1,linen scents_Le Blanc_Unknown,1.008787
2,linen starches_Le Blanc_Unknown,0.993371
3,guest towels_Sferra_White/Ivory,0.901663
4,"bedding,pillow cases_Sferra_White / White",0.745242
5,table protectors_Gracious Style_White,0.717978
6,"duvet covers,bedding_Sferra_White / White",0.700158
7,napkins_Sferra_Raspberry,0.674275
8,fitted sheets_Sferra_Ivory,0.626471
9,pillow cases_Kassatex_White,0.618171
